In [ ]:
import geopandas as gpd


In [ ]:
# Runtime 10m, loading in the NFIxAWI overlayed datasets
datasets = {}
for year in range(2022, 2023):
    datasets[f'nfi_awi_overlay_{year}'] = gpd.read_file(f'../data/processed/gb_nfi_awi_overlay_{year}.geojson')

In [ ]:
# Runtime 20m, loading in the NFI overlayed datasets
for year in range(2012, 2023):
    datasets[f'gb_nfi_dataset_{year}'] = gpd.read_file(f'../data/processed/gb_nfi_dataset_{year}.geojson')

In [ ]:
# Runtime 45s, loading the AWI dataset
datasets['gb_awi_dataset'] = gpd.read_file('../data/processed/gb_awi_dataset.geojson')

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from shapely.geometry import box

app = Flask(__name__)
CORS(app)

@app.route('/calculate_areas', methods=['GET'])
def calculate_areas():
    dataset = datasets[request.args.get('dataset')]
    bounds = [float(b) for b in request.args.get('bounds').split(',')]
    type = request.args.get('type')
    bbox = box(*bounds)

    print(f'type: {type}')

    sindex = dataset.sindex
    selected_indices = list(sindex.intersection(bbox.bounds))
    selected_data = dataset.iloc[selected_indices]

    areas = selected_data.groupby(type)['area_ha'].sum()
    return jsonify(areas.to_dict())

if __name__ == '__main__':
    app.run(port=3901, debug=False)